In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from setting import *
from prehandle.Dataset import MyDataset
from tqdm import tqdm
from trans_model import TransformerEncoder, train, predict, evaluate


In [2]:
# 生成数据集
from pathlib2 import Path
data_path = r"C:\Users\sivan\Downloads\LogdiffTest\difference data\runtimeapi2"
data_path = Path(data_path)
dataset = MyDataset(data_path, padding_length=50,stride=1)
import pickle
with open('dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

100%|██████████| 1/1 [00:05<00:00,  5.20s/it]


In [3]:
import pickle
dataset = pickle.load(open('dataset.pkl', 'rb'))
dataset.vocab_size

141

In [4]:
# hyperparameters
embedding_dim = dataset.model_dim
seq_length = dataset.padding_length * 2
num_classes = dataset.vocab_size
hidden_dim = num_classes // 2
num_layers = 3
batch_size = 128
lr = 0.001
epochs = 1
loader = DataLoader(dataset.data, batch_size=batch_size, shuffle=True)

In [7]:
# initialize model
model = TransformerEncoder(embedding_dim, hidden_dim, num_classes, num_layers, seq_length)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = model.to(device)
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [8]:
loss_list = train(model, loader, optimizer, criterion, epochs, device)

Epoch: 1


  0%|          | 0/863 [00:00<?, ?it/s]


IndexError: Target 9377 is out of bounds.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()

In [48]:
print("K=1")
predictions, y_true = predict(model,loader,device,K=1)
accuracy, abnormal = evaluate(y_true, predictions)
print("accuracy: {:.2f}%".format(accuracy))

print("K=2")
predictions, y_true = predict(model,loader,device,K=2)
accuracy, abnormal = evaluate(y_true, predictions)
print("accuracy: {:.2f}%".format(accuracy))

print("K=3")
predictions, y_true = predict(model,loader,device,K=3)
accuracy, abnormal = evaluate(y_true, predictions)
print("accuracy: {:.2f}%".format(accuracy))

K=1


100%|██████████| 912/912 [00:38<00:00, 23.89it/s]


accuracy: 36.65%
K=2


100%|██████████| 912/912 [00:37<00:00, 24.18it/s]


accuracy: 47.34%
K=3


100%|██████████| 912/912 [00:37<00:00, 24.18it/s]

accuracy: 50.66%


In [42]:
loader = DataLoader(dataset.data, batch_size=128, shuffle=False)
predictions, y_true = predict(model,loader,device,K=2)
accuracy, abnormal = evaluate(y_true, predictions)

100%|██████████| 863/863 [00:36<00:00, 23.93it/s]


In [43]:
# 将间断的不正常数据合并为连续的不正常区域
def get_consistent_abnormal(abnormal):
    consistent_abnormal = []
    left,right = abnormal[0][0], abnormal[0][0] + 1
    for i in range(1, len(abnormal)):
        if abnormal[i][0] == right:
            right += 1
        else:
            consistent_abnormal.append((left, right))
            left, right = abnormal[i][0], abnormal[i][0] + 1
    consistent_abnormal.append((left, right))
    return consistent_abnormal
consistent_abnormal = get_consistent_abnormal(abnormal)
consistent_abnormal


[(0, 7),
 (8, 12),
 (13, 14),
 (15, 17),
 (18, 20),
 (22, 24),
 (27, 28),
 (29, 31),
 (36, 37),
 (58, 59),
 (65, 66),
 (67, 68),
 (73, 76),
 (77, 80),
 (82, 84),
 (164, 167),
 (168, 202),
 (205, 206),
 (211, 214),
 (215, 216),
 (217, 220),
 (263, 264),
 (267, 273),
 (274, 275),
 (289, 290),
 (355, 357),
 (364, 370),
 (372, 376),
 (381, 386),
 (388, 389),
 (394, 396),
 (397, 401),
 (413, 414),
 (421, 422),
 (3884, 3885),
 (3939, 3945),
 (3946, 3947),
 (4306, 4308),
 (4310, 4312),
 (4313, 4317),
 (5228, 5229),
 (22862, 22863),
 (23483, 23484),
 (23507, 23508),
 (23510, 23511),
 (23577, 23578),
 (23583, 23587),
 (24193, 24195),
 (36394, 36395),
 (36401, 36402),
 (37816, 37817),
 (38981, 38982),
 (38984, 38985),
 (39030, 39031),
 (39086, 39087),
 (39096, 39097),
 (39098, 39099),
 (39101, 39103),
 (39134, 39135),
 (39162, 39163),
 (39174, 39175),
 (39181, 39183),
 (39187, 39189),
 (39195, 39196),
 (39228, 39229),
 (39231, 39232),
 (39235, 39236),
 (39253, 39255),
 (39256, 39257),
 (39262, 3

In [44]:
from pathlib2 import Path
data_path = r"C:\Users\sivan\Downloads\LogdiffTest\difference data\runtime api1"
data_path = Path(data_path)
dataset = MyDataset(data_path, padding_length=50,stride=1)
loader = DataLoader(dataset.data, batch_size=128, shuffle=False)

100%|██████████| 1/1 [00:05<00:00,  5.31s/it]


In [61]:
predictions, y_true = predict(model,loader,device,K=10)
accuracy, abnormal = evaluate(y_true, predictions)
accuracy

100%|██████████| 912/912 [00:38<00:00, 23.95it/s]


64.52605944193864

47.336603268792125

In [53]:
abnormal[-500:]

[(116110, 14119, [6333, 14839]),
 (116111, 14119, [6333, 9377]),
 (116112, 14118, [6333, 9377]),
 (116113, 14119, [6333, 9377]),
 (116114, 14119, [6333, 9377]),
 (116115, 14119, [6333, 14839]),
 (116116, 14117, [6333, 14839]),
 (116117, 1290, [6333, 14839]),
 (116118, 14118, [6333, 14839]),
 (116119, 14118, [6333, 14839]),
 (116120, 14119, [6333, 9377]),
 (116121, 14119, [6333, 14839]),
 (116122, 14118, [6333, 9377]),
 (116123, 14119, [9377, 6333]),
 (116124, 14119, [6333, 14839]),
 (116125, 14118, [9377, 6333]),
 (116126, 14119, [6333, 14839]),
 (116127, 14119, [6333, 9377]),
 (116128, 14118, [11194, 9377]),
 (116129, 14119, [6333, 9377]),
 (116130, 14119, [6333, 9377]),
 (116131, 14118, [6333, 9377]),
 (116132, 14119, [6333, 14839]),
 (116133, 14119, [6333, 14839]),
 (116134, 14118, [6333, 14839]),
 (116135, 14119, [6333, 14839]),
 (116136, 14119, [6333, 14839]),
 (116137, 14118, [6333, 14839]),
 (116138, 14119, [6333, 14839]),
 (116139, 14119, [6333, 9377]),
 (116140, 14118, [6333, 

In [52]:
len(abnormal)

61415

In [1]:
import numpy as np
from setting import api_dic_path
api_dic = np.load(api_dic_path, allow_pickle=True).item()

In [9]:
index2api = {int(v[0]):k for k,v in api_dic.items()}

In [57]:
index2api[6333]

'HeapSetInformation'

In [20]:
import pandas as pd
path = r"C:\Users\sivan\Downloads\LogdiffTest\difference data\runtimeapi2\runtimeAPI2.csv"
from log import ApiLog
log = ApiLog(path)

In [24]:
prototype = log.df["Prototype"].tolist()

In [27]:
from typing import Counter
cnt = Counter(prototype)

In [60]:
index_list = [6333,9377,11194]
for index in index_list:
    print(index, index2api[index], cnt[index2api[index]])

6333 HeapSetInformation 0
9377 MultinetGetConnectionPerformanceA 0
11194 ReadFileEx 0


In [30]:
cnt.most_common(10)

[('MultiByteToWideChar', 40560),
 ('TlsGetValue', 12950),
 ('CompareStringW', 9796),
 ('WakeAllConditionVariable', 8131),
 ('TlsSetValue', 6488),
 ('SetEvent', 5234),
 ('WaitForMultipleObjectsEx', 4595),
 ('ResetEvent', 4289),
 ('HeapReAlloc', 4237),
 ('SleepConditionVariableCS', 2500)]